<a href="https://colab.research.google.com/github/BRAHIMLOUARDI/movie-web-app/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation file
path_to_data = '/content/fra.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs=pairs[0:192341]



In [43]:
# list_of_lists = [[1,2,3],[4,5,6],[7,8,9]]

# import csv

# with open("out.csv", "w") as f:
#     wr = csv.writer(f)
   
#     wr.writerows(exppairs)



In [59]:
exppairs=pairs[0:10]
file = open("sample.py", "w+")

# Saving the array in a text file
for pair in exppairs :
   file.write(str(pair))
   file.write("\n")
file.close()





In [13]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "¡" + '¿'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [14]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [ ]:

english_sentences = [clean_sentence(pair[0]) for pair in pairs]
french_sentences = [clean_sentence(pair[1]) for pair in pairs]

# Tokenize words
fra_text_tokenized, fra_text_tokenizer = tokenize(french_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length french sentence: {}'.format(len(max(fra_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))


# Check language length
french_vocab = len(fra_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("Spanish vocabulary is of {} unique words".format(french_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

In [91]:
max_french_len = int(len(max(fra_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

fra_pad_sentence = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

fra_pad_sentence1 = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
eng_pad_sentence1 = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# Reshape data
fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [92]:
input_sequence = Input(shape=(max_french_len,))
embedding = Embedding(input_dim=french_vocab, output_dim=128,)(input_sequence)

In [94]:
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)


In [95]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 55)]              0         
                                                                 
 embedding (Embedding)       (None, 55, 128)           4551936   
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVector  (None, 47, 64)           0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 47, 64)            33024     
                                                                 
 time_distributed (TimeDistr  (None, 47, 15850)        1030250   
 ibuted)                                                     

In [98]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

In [99]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=30, epochs=100,callbacks=[checkpoint])


Epoch 1/100
 276/6412 [>.............................] - ETA: 1:03:18 - loss: 1.6449 - accuracy: 0.8709

KeyboardInterrupt: ignored

In [ ]:
enc_dec_model.save('my_model.h5')

In [ ]:

def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 14
print("The english sentence is: {}".format(english_sentences[index]))
print("The spanish sentence is: {}".format(french_sentences[index]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(, eng_text_tokenizer))

In [ ]:

def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 14
print("The english sentence is: {}".format(english_sentences[index]))
print("The spanish sentence is: {}".format(french_sentences[index]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(, eng_text_tokenizer))

In [ ]:

def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 14
print("The english sentence is: {}".format(english_sentences[index]))
print("The spanish sentence is: {}".format(french_sentences[index]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(fra_pad_sentence[index:index+1])[0], eng_text_tokenizer))
print("rfrif")
print(fra_pad_sentence[index:index+1])






In [ ]:
new_model = tf.keras.models.load_model('my_model.h5')
index = 15


print(fra_text_tokenized)

print(fra_text_tokenizer.word_index)
print(fra_text_tokenized)


# fra_pad_sentence = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
# eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# fra_pad_sentence1 = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
# eng_pad_sentence1 = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# # Reshape data
# fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
# eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)


print(fra_pad_sentence[0])



In [ ]:
new_model = tf.keras.models.load_model('my_model.h5')


print(fra_text_tokenized)

print(fra_text_tokenizer.word_index)
print(fra_text_tokenized)


# fra_pad_sentence = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
# eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# fra_pad_sentence1 = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
# eng_pad_sentence1 = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# # Reshape data
# fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
# eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)


In [ ]:
from numpy.core.fromnumeric import shape
fra_text = pad_sequences([[1,4,2404]], max_french_len, padding = "post")

fra_text = fra_text.reshape(*fra_text.shape, 1)
print(fra_text)
index = 30
print("The english sentence is: {}".format(english_sentences[index]))
print("The spanish sentence is: {}".format(french_sentences[index]))
print('The predicted sentence is :')
hhh,jjjjj=enc_dec_model.predict(fra_pad_sentence[index:index+1])[0], eng_text_tokenizer



print(np.argmax(hhh[1]))

print(eng_text_tokenizer.word_index)

experment


In [ ]:
#exp

input_sequence1 = Input(shape=(12,))
embedding1 = Embedding(input_dim=7188, output_dim=128,)(input_sequence1)

model1 = Model(inputs=input_sequence1 , outputs=embedding1, name="mnist_model")
model1.compile('rmsprop', 'mse')


input_data = np.array([[1,2,4,5,18,0,0,0,4,10,12,13]])

output_array = model1.predict(input_data)
print(output_array[0][0])


In [ ]:
# for layer in model1.layers: print(layer.get_config(), layer.get_weights())
#print(model1.layers[1].get_weights()[0].shape)

(7188, 128)


In [ ]:
#exp
lstm = LSTM(64, return_sequences=False)(embedding1)

lstmrepeatvector = RepeatVector(6)(lstm)
lstmdecoder = LSTM(64, return_sequences=True, dropout=0.2)(lstmrepeatvector)
lstmlogits = TimeDistributed(Dense(3733))(lstmdecoder)
modellstm = Model(inputs=input_sequence1 , outputs=lstmlogits, name="mnist_model")

In [ ]:
#exp
modellstm.compile('rmsprop', 'mse')

In [ ]:
#EXP
output_array = modellstm.predict(input_data)
reshape_out=np.reshape(output_array[0][0],(3733,-1))
print(reshape_out.shape)


(3733, 1)
